<a href="https://colab.research.google.com/github/milad9595naeimi/Export_Technology_Database/blob/main/Technological_Specialization_IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. from google.colab import drive:

This line imports the drive module from the google.colab package. The google.colab package provides various utilities to interact with Google Colab, and the drive module specifically deals with operations related to Google Drive.

2. drive.mount('/content/drive/'):

This line mounts Google Drive to the specified path in the Colab environment, allowing you to access files stored in your Google Drive as if they were part of the local file system on the Colab instance.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


1. !pip install google-cloud-bigquery:


The actual command pip install google-cloud-bigquery is used to install the Google Cloud BigQuery client library for Python. In a Jupyter notebook or Colab, this line installs the necessary package so you can interact with BigQuery.

2. import os:

This line imports the os module, which provides a way of using operating system-dependent functionality like reading or writing to the file system, environment variables, etc.

3. from google.cloud import bigquery:
This line imports the bigquery module from the google.cloud package, which contains the client library for interacting with BigQuery.

4. import pandas as pd:
This line imports the pandas library, which is a powerful data manipulation and analysis library in Python. It's often used for handling data structures like DataFrames.

In [ ]:
#!pip install google-cloud-bigquery
import os
from google.cloud import bigquery
import pandas as pd

1. !pip install google-cloud-bigquery:

The ! at the beginning is used in Jupyter notebooks and Google Colab to execute shell commands directly from a code cell.
The pip install google-cloud-bigquery command installs the Google Cloud BigQuery client library for Python, which is necessary for interacting with BigQuery services.

2. !pip install pyarrow:
The pip install pyarrow command installs the pyarrow library, which provides a Python interface to the Apache Arrow framework. Arrow is used for efficient in-memory data representation and is particularly useful for reading and writing data quickly.

In [ ]:
!pip install google-cloud-bigquery
!pip install pyarrow


1. import os:
This line imports the os module, which provides functions to interact with the operating system, including manipulating environment variables.

2. key_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/patents-426102-ce4b63cd244f.json':
This line defines a variable key_file_path that holds the path to the service account key JSON file. This file is assumed to be uploaded to Google Drive and mounted in Colab.

3. os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file_path:
This line sets the GOOGLE_APPLICATION_CREDENTIALS environment variable to the path of the service account key file. This environment variable is used by Google Cloud client libraries to authenticate API requests.

In [ ]:
import os

# Assuming the JSON file is uploaded to the root directory in Colab
key_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/patents-426102-ce4b63cd244f.json'

# Set up Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file_path


1. from google.cloud import bigquery:

This line imports the bigquery module from the google.cloud package. This module provides the functionality needed to interact with Google BigQuery, such as running queries and managing datasets.

2. import pandas as pd:

This line imports the pandas library, which is a powerful tool for data manipulation and analysis in Python. It provides data structures like DataFrames for handling tabular data.

3. import re:

This line imports the re module, which provides support for regular expressions in Python. Regular expressions are used for matching patterns in strings, which can be useful for data cleaning and validation tasks.

4. client = bigquery.Client():

This line initializes a BigQuery client object. The Client class is used to interact with BigQuery, allowing you to run SQL queries, retrieve results, and manage datasets and tables.

In [ ]:
from google.cloud import bigquery
import pandas as pd
import re  # Make sure to import the re module for regular expressions

# Initialize BigQuery client
client = bigquery.Client()


Retrieving data from Google Patents Public Datasets Using BigQuery. Only Change the codes for ipc and it works

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
client = bigquery.Client()

# Combined SQL query with proper filtering and aggregation
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
ipc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT ipc.code, ' | ') AS ipc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(ipc) AS ipc
  WHERE
    ipc.code LIKE "B64B%" OR ipc.code LIKE "B64C%" OR ipc.code LIKE "B64D%" OR ipc.code LIKE "B64F%" OR ipc.code LIKE "B64G%"
  GROUP BY
    publication_number
  HAVING
    COUNT(ipc.code) > 0
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.ipc_codes
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  ipc_data AS c
ON
  p.publication_number = c.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 100;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
print(df)


Add "assignee_country" to the previous line of code to have a new column of "assignee_country" in addition to other columns.This code is designed to retrieve patent data from Google BigQuery, specifically focusing on US patents published between 1989 and 2008. It aggregates inventor countries, ipc codes, and assignee countries for each patent, ensuring that the inventor and ipc data meet specified filtering criteria. The final dataset includes publication numbers, publication years, inventor countries, ipc codes, and assignee countries, providing a comprehensive view of the relevant patent information.











1. Detailed Explanation of the Query
Common Table Expressions (CTEs)
inventor_data CTE:

Purpose: To collect and aggregate inventor country codes for each patent.
Details:
""SELECT
  publication_number,
  STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(inventor_harmonized) AS inventor_harmonized
WHERE
  inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
GROUP BY
  publication_number
HAVING
  COUNT(inventor_harmonized.country_code) > 0""

a) UNNEST(inventor_harmonized): Flattens the nested inventor_harmonized array.
b) STRING_AGG(inventor_harmonized.country_code, ' | '): Aggregates inventor country codes separated by |.
c) WHERE clause: Filters inventors by specified country codes.
d) HAVING COUNT(...) > 0: Ensures only patents with at least one valid inventor country code are included.



2. ipc_data CTE:

Purpose: To collect and aggregate ipc codes for each patent.
Details:
""SELECT
  publication_number,
  STRING_AGG(DISTINCT ipc.code, ' | ') AS ipc_codes
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(ipc) AS ipc
WHERE
  ipc.code LIKE "B64B%" OR ipc.code LIKE "B64C%" OR ipc.code LIKE "B64D%" OR ipc.code LIKE "B64F%" OR ipc.code LIKE "B64G%"
GROUP BY
  publication_number
HAVING
  COUNT(ipc.code) > 0""
a) UNNEST(ipc): Flattens the nested ipc array.
b) STRING_AGG(DISTINCT ipc.code, ' | '): Aggregates distinct ipc codes separated by |.
c) WHERE clause: Filters ipc codes by specified patterns.
d) HAVING COUNT(...) > 0: Ensures only patents with at least one valid ipc code are included.



3. assignee_data CTE:

Purpose: To collect and aggregate assignee country codes for each patent without filtering by specific country codes.
Details:
""SELECT
  publication_number,
  STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(assignee_harmonized) AS assignee_harmonized
GROUP BY
  publication_number""
a) UNNEST(assignee_harmonized): Flattens the nested assignee_harmonized array.
b) STRING_AGG(assignee_harmonized.country_code, ' | '): Aggregates assignee country codes separated by |.



4. Main Query
Purpose: To join the results from the CTEs and include additional filters to generate the final dataset.
Details:
""SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.ipc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  ipc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 100;""
a) SELECT Clause: Retrieves the publication number, publication year, inventor countries, ipc codes, and assignee countries.
b) JOIN Clauses: Joins the main table with the CTEs on publication_number:
b.1) INNER JOIN with inventor_data and ipc_data to ensure only patents with both valid inventor countries and ipc codes are included.
b.2) LEFT JOIN with assignee_data to include all patents, even those without assignee countries.
c) WHERE Clause: Filters to include only US patents and publication years between 1989 and 2008.
d) LIMIT Clause: Limits the results to 100 entries for simplicity.



5. Running the Query and Converting to DataFrame
Purpose: Execute the query, retrieve the results, convert them to a pandas DataFrame, and display the DataFrame.
Details:
a) client.query(query): Executes the SQL query.
b) query_job.result(): Retrieves the query results.
c) results.to_dataframe(): Converts the results to a pandas DataFrame.

In [ ]:
# SQL query to only retrieve data
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
ipc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT ipc.code, ' | ') AS ipc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(ipc) AS ipc
  WHERE
    ipc.code LIKE "B64B%" OR ipc.code LIKE "B64C%" OR ipc.code LIKE "B64D%" OR ipc.code LIKE "B64F%" OR ipc.code LIKE "B64G%" OR
    ipc.code LIKE "A61K%" OR ipc.code LIKE "C07K%" OR ipc.code LIKE "A61P%" OR ipc.code LIKE "C12N%" OR ipc.code LIKE "C07D%" OR
    ipc.code LIKE "C12P%" OR ipc.code LIKE "C07H%" OR ipc.code LIKE "C12Q%" OR ipc.code LIKE "C07J%" OR ipc.code LIKE "B41J%" OR
    ipc.code LIKE "G05F%" OR ipc.code LIKE "G06G%" OR ipc.code LIKE "G06T%" OR ipc.code LIKE "G07G%" OR ipc.code LIKE "H03K%" OR
    ipc.code LIKE "B41K%" OR ipc.code LIKE "G06C%" OR ipc.code LIKE "G06J%" OR ipc.code LIKE "G07B%" OR ipc.code LIKE "G09D%" OR
    ipc.code LIKE "H03L%" OR ipc.code LIKE "B43M%" OR ipc.code LIKE "G06D%" OR ipc.code LIKE "G06K%" OR ipc.code LIKE "G07C%" OR
    ipc.code LIKE "G09G%" OR ipc.code LIKE "G02F%" OR ipc.code LIKE "G06E%" OR ipc.code LIKE "G06M%" OR ipc.code LIKE "G07D%" OR
    ipc.code LIKE "G10L%" OR ipc.code LIKE "G03G%" OR ipc.code LIKE "G06F%" OR ipc.code LIKE "G06N%" OR ipc.code LIKE "G07F%" OR
    ipc.code LIKE "G11B%" OR ipc.code LIKE "B81B%" OR ipc.code LIKE "G11C%" OR ipc.code LIKE "H01L%" OR ipc.code LIKE "H03B%" OR
    ipc.code LIKE "H03H%" OR ipc.code LIKE "H04J%" OR ipc.code LIKE "H04Q%" OR ipc.code LIKE "B81C%" OR ipc.code LIKE "H01C%" OR
    ipc.code LIKE "H01P%" OR ipc.code LIKE "H03C%" OR ipc.code LIKE "H03J%" OR ipc.code LIKE "H04K%" OR ipc.code LIKE "H04R%" OR
    ipc.code LIKE "G03H%" OR ipc.code LIKE "H01F%" OR ipc.code LIKE "H01Q%" OR ipc.code LIKE "H03D%" OR ipc.code LIKE "H03M%" OR
    ipc.code LIKE "H04L%" OR ipc.code LIKE "H04S%" OR ipc.code LIKE "G09B%" OR ipc.code LIKE "H01G%" OR ipc.code LIKE "H01S%" OR
    ipc.code LIKE "H03F%" OR ipc.code LIKE "H04B%" OR ipc.code LIKE "H04M%" OR ipc.code LIKE "H05K%" OR ipc.code LIKE "G09C%" OR
    ipc.code LIKE "H01J%" OR ipc.code LIKE "H02J%" OR ipc.code LIKE "H03G%" OR ipc.code LIKE "H04H%" OR ipc.code LIKE "H04N%" OR
    ipc.code LIKE "A61B%" OR ipc.code LIKE "A61J%" OR ipc.code LIKE "B04B%" OR ipc.code LIKE "G01F%" OR ipc.code LIKE "G01N%" OR
    ipc.code LIKE "G02C%" OR ipc.code LIKE "G04D%" OR ipc.code LIKE "G12B%" OR ipc.code LIKE "A61C%" OR ipc.code LIKE "A61L%" OR
    ipc.code LIKE "C12M%" OR ipc.code LIKE "G01H%" OR ipc.code LIKE "G01R%" OR ipc.code LIKE "G03B%" OR ipc.code LIKE "G04F%" OR
    ipc.code LIKE "G21G%" OR ipc.code LIKE "A61D%" OR ipc.code LIKE "A61M%" OR ipc.code LIKE "F15C%" OR ipc.code LIKE "G01J%" OR
    ipc.code LIKE "G01S%" OR ipc.code LIKE "G03D%" OR ipc.code LIKE "G04G%" OR ipc.code LIKE "G21K%" OR ipc.code LIKE "A61F%" OR
    ipc.code LIKE "A61N%" OR ipc.code LIKE "G01B%" OR ipc.code LIKE "G01K%" OR ipc.code LIKE "G01T%" OR ipc.code LIKE "G03F%" OR
    ipc.code LIKE "G05B%" OR ipc.code LIKE "H05G%" OR ipc.code LIKE "A61G%" OR ipc.code LIKE "A62B%" OR ipc.code LIKE "G01C%" OR
    ipc.code LIKE "G01L%" OR ipc.code LIKE "G01W%" OR ipc.code LIKE "G04B%" OR ipc.code LIKE "G08C%" OR ipc.code LIKE "A61H%" OR
    ipc.code LIKE "B01L%" OR ipc.code LIKE "G01D%" OR ipc.code LIKE "G01M%" OR ipc.code LIKE "G02B%" OR ipc.code LIKE "G04C%" OR
    ipc.code LIKE "G09F%" OR ipc.code LIKE "A01N%" OR ipc.code LIKE "C01D%" OR ipc.code LIKE "C06B%" OR ipc.code LIKE "C08F%" OR
    ipc.code LIKE "C09F%" OR ipc.code LIKE "C10K%" OR ipc.code LIKE "C25B%" OR ipc.code LIKE "G03C%" OR ipc.code LIKE "A62D%" OR
    ipc.code LIKE "C01F%" OR ipc.code LIKE "C06C%" OR ipc.code LIKE "C08G%" OR ipc.code LIKE "C09G%" OR ipc.code LIKE "C10M%" OR
    ipc.code LIKE "D01C%" OR ipc.code LIKE "G21F%" OR ipc.code LIKE "B01J%" OR ipc.code LIKE "C01G%" OR ipc.code LIKE "C06D%" OR
    ipc.code LIKE "C08H%" OR ipc.code LIKE "C09H%" OR ipc.code LIKE "C11B%" OR ipc.code LIKE "D01F%" OR ipc.code LIKE "B09B%" OR
    ipc.code LIKE "C02F%" OR ipc.code LIKE "C07B%" OR ipc.code LIKE "C08J%" OR ipc.code LIKE "C09J%" OR ipc.code LIKE "C11C%" OR
    ipc.code LIKE "D06L%" OR ipc.code LIKE "B09C%" OR ipc.code LIKE "C05B%" OR ipc.code LIKE "C07C%" OR ipc.code LIKE "C08K%" OR
    ipc.code LIKE "C09K%" OR ipc.code LIKE "C11D%" OR ipc.code LIKE "F17C%" OR ipc.code LIKE "B27K%" OR ipc.code LIKE "C05C%" OR
    ipc.code LIKE "C07F%" OR ipc.code LIKE "C08L%" OR ipc.code LIKE "C10B%" OR ipc.code LIKE "C12S%" OR ipc.code LIKE "F17D%" OR
    ipc.code LIKE "B29B%" OR ipc.code LIKE "C05D%" OR ipc.code LIKE "C07G%" OR ipc.code LIKE "C09B%" OR ipc.code LIKE "C10C%" OR
    ipc.code LIKE "C14C%" OR ipc.code LIKE "F25J%" OR ipc.code LIKE "C01B%" OR ipc.code LIKE "C05F%" OR ipc.code LIKE "C08B%" OR
        ipc.code LIKE "C08C%" OR ipc.code LIKE "C09D%" OR ipc.code LIKE "C10J%" OR ipc.code LIKE "C23G%" OR ipc.code LIKE "F42D%" OR
    ipc.code LIKE "B60M%" OR ipc.code LIKE "F21M%" OR ipc.code LIKE "G08B%" OR ipc.code LIKE "H01H%" OR ipc.code LIKE "H02B%" OR
    ipc.code LIKE "H02P%" OR ipc.code LIKE "B61L%" OR ipc.code LIKE "F21P%" OR ipc.code LIKE "G08G%" OR ipc.code LIKE "H01K%" OR
    ipc.code LIKE "H02H%" OR ipc.code LIKE "H05C%" OR ipc.code LIKE "F21H%" OR ipc.code LIKE "F21Q%" OR ipc.code LIKE "G10K%" OR
    ipc.code LIKE "H01M%" OR ipc.code LIKE "H02K%" OR ipc.code LIKE "F21K%" OR ipc.code LIKE "F21S%" OR ipc.code LIKE "G21C%" OR
    ipc.code LIKE "H01R%" OR ipc.code LIKE "H02M%" OR ipc.code LIKE "F21L%" OR ipc.code LIKE "F21V%" OR ipc.code LIKE "G21D%" OR
    ipc.code LIKE "H01T%" OR ipc.code LIKE "H02N%" OR ipc.code LIKE "A01B%" OR ipc.code LIKE "B01D%" OR ipc.code LIKE "B23D%" OR
    ipc.code LIKE "B31F%" OR ipc.code LIKE "B68F%" OR ipc.code LIKE "D06B%" OR ipc.code LIKE "F04B%" OR ipc.code LIKE "F23N%" OR
    ipc.code LIKE "F41J%" OR ipc.code LIKE "A01C%" OR ipc.code LIKE "B01F%" OR ipc.code LIKE "B23F%" OR ipc.code LIKE "B41B%" OR
    ipc.code LIKE "C10F%" OR ipc.code LIKE "D06F%" OR ipc.code LIKE "F04C%" OR ipc.code LIKE "F23R%" OR ipc.code LIKE "F42C%" OR
    ipc.code LIKE "A01D%" OR ipc.code LIKE "B02B%" OR ipc.code LIKE "B23G%" OR ipc.code LIKE "B41C%" OR ipc.code LIKE "C12L%" OR
    ipc.code LIKE "D06G%" OR ipc.code LIKE "F04D%" OR ipc.code LIKE "F24B%" OR ipc.code LIKE "G01G%" OR ipc.code LIKE "A01F%" OR
    ipc.code LIKE "B02C%" OR ipc.code LIKE "B23H%" OR ipc.code LIKE "B41D%" OR ipc.code LIKE "C13C%" OR ipc.code LIKE "D06H%" OR
    ipc.code LIKE "F04F%" OR ipc.code LIKE "F24C%" OR ipc.code LIKE "G21J%" OR ipc.code LIKE "A01G%" OR ipc.code LIKE "B03B%" OR
    ipc.code LIKE "B23K%" OR ipc.code LIKE "B41F%" OR ipc.code LIKE "C13D%" OR ipc.code LIKE "D21B%" OR ipc.code LIKE "F15B%" OR
    ipc.code LIKE "F24D%" OR ipc.code LIKE "H05B%" OR ipc.code LIKE "A01J%" OR ipc.code LIKE "B03C%" OR ipc.code LIKE "B23P%" OR
    ipc.code LIKE "B41G%" OR ipc.code LIKE "C13G%" OR ipc.code LIKE "D21D%" OR ipc.code LIKE "F16C%" OR ipc.code LIKE "F24F%" OR
    ipc.code LIKE "H05F%" OR ipc.code LIKE "A01K%" OR ipc.code LIKE "B03D%" OR ipc.code LIKE "B23Q%" OR ipc.code LIKE "B41L%" OR
    ipc.code LIKE "C13H%" OR ipc.code LIKE "D21F%" OR ipc.code LIKE "F16D%" OR ipc.code LIKE "F24H%" OR ipc.code LIKE "H05H%" OR
    ipc.code LIKE "A01M%" OR ipc.code LIKE "B04C%" OR ipc.code LIKE "B24B%" OR ipc.code LIKE "B41N%" OR ipc.code LIKE "C14B%" OR
    ipc.code LIKE "D21G%" OR ipc.code LIKE "F16F%" OR ipc.code LIKE "F25B%" OR ipc.code LIKE "A21B%" OR ipc.code LIKE "B05B%" OR
    ipc.code LIKE "B24C%" OR ipc.code LIKE "B42B%" OR ipc.code LIKE "C23C%" OR ipc.code LIKE "E01C%" OR ipc.code LIKE "F16G%" OR
    ipc.code LIKE "F25C%" OR ipc.code LIKE "A21C%" OR ipc.code LIKE "B05C%" OR ipc.code LIKE "B25D%" OR ipc.code LIKE "B42C%" OR
    ipc.code LIKE "D01B%" OR ipc.code LIKE "E02D%" OR ipc.code LIKE "F16H%" OR ipc.code LIKE "F25D%" OR ipc.code LIKE "A22B%" OR
    ipc.code LIKE "B05D%" OR ipc.code LIKE "B25J%" OR ipc.code LIKE "B44B%" OR ipc.code LIKE "D01D%" OR ipc.code LIKE "E02F%" OR
    ipc.code LIKE "F16K%" OR ipc.code LIKE "F26B%" OR ipc.code LIKE "A22C%" OR ipc.code LIKE "B06B%" OR ipc.code LIKE "B26D%" OR
    ipc.code LIKE "B61B%" OR ipc.code LIKE "D01G%" OR ipc.code LIKE "E06C%" OR ipc.code LIKE "F16M%" OR ipc.code LIKE "F27B%" OR
    ipc.code LIKE "A23N%" OR ipc.code LIKE "B07B%" OR ipc.code LIKE "B26F%" OR ipc.code LIKE "B63G%" OR ipc.code LIKE "D01H%" OR
    ipc.code LIKE "E21B%" OR ipc.code LIKE "F16N%" OR ipc.code LIKE "F28B%" OR ipc.code LIKE "A24C%" OR ipc.code LIKE "B07C%" OR
    ipc.code LIKE "B27B%" OR ipc.code LIKE "B65B%" OR ipc.code LIKE "D02G%" OR ipc.code LIKE "E21C%" OR ipc.code LIKE "F22D%" OR
    ipc.code LIKE "F28C%" OR ipc.code LIKE "A41H%" OR ipc.code LIKE "B08B%" OR ipc.code LIKE "B27C%" OR ipc.code LIKE "B65C%" OR
    ipc.code LIKE "D02H%" OR ipc.code LIKE "E21D%" OR ipc.code LIKE "F23B%" OR ipc.code LIKE "F28D%" OR ipc.code LIKE "A42C%" OR
    ipc.code LIKE "B21B%" OR ipc.code LIKE "B27F%" OR ipc.code LIKE "B65G%" OR ipc.code LIKE "D02J%" OR ipc.code LIKE "E21F%" OR
    ipc.code LIKE "F23C%" OR ipc.code LIKE "F28F%" OR ipc.code LIKE "A43D%" OR ipc.code LIKE "B21D%" OR ipc.code LIKE "B27J%" OR
    ipc.code LIKE "B65H%" OR ipc.code LIKE "D03C%" OR ipc.code LIKE "F01B%" OR ipc.code LIKE "F23D%" OR ipc.code LIKE "F28G%" OR
    ipc.code LIKE "A45D%" OR ipc.code LIKE "B21F%" OR ipc.code LIKE "B27L%" OR ipc.code LIKE "B66B%" OR ipc.code LIKE "D03D%" OR
    ipc.code LIKE "F01C%" OR ipc.code LIKE "F23G%" OR ipc.code LIKE "F41A%" OR ipc.code LIKE "A47G%" OR ipc.code LIKE "B21H%" OR
    ipc.code LIKE "B28D%" OR ipc.code LIKE "B66C%" OR ipc.code LIKE "D03J%" OR ipc.code LIKE "F01D%" OR ipc.code LIKE "F23H%" OR
    ipc.code LIKE "F41B%" OR ipc.code LIKE "A47J%" OR ipc.code LIKE "B21J%" OR ipc.code LIKE "B30B%" OR ipc.code LIKE "B66D%" OR
        ipc.code LIKE "D04B%" OR ipc.code LIKE "F03B%" OR ipc.code LIKE "F23J%" OR ipc.code LIKE "F41C%" OR ipc.code LIKE "A47L%" OR
    ipc.code LIKE "B22C%" OR ipc.code LIKE "B31B%" OR ipc.code LIKE "B66F%" OR ipc.code LIKE "D04C%" OR ipc.code LIKE "F03C%" OR
    ipc.code LIKE "F23K%" OR ipc.code LIKE "F41F%" OR ipc.code LIKE "A62C%" OR ipc.code LIKE "B23B%" OR ipc.code LIKE "B31C%" OR
    ipc.code LIKE "B67B%" OR ipc.code LIKE "D05B%" OR ipc.code LIKE "F03D%" OR ipc.code LIKE "F23L%" OR ipc.code LIKE "F41G%" OR
    ipc.code LIKE "B01B%" OR ipc.code LIKE "B23C%" OR ipc.code LIKE "B31D%" OR ipc.code LIKE "B67C%" OR ipc.code LIKE "D05C%" OR
    ipc.code LIKE "F03G%" OR ipc.code LIKE "F23M%" OR ipc.code LIKE "F41H%" OR ipc.code LIKE "B60B%" OR ipc.code LIKE "B60K%" OR
    ipc.code LIKE "B60R%" OR ipc.code LIKE "F01L%" OR ipc.code LIKE "F02D%" OR ipc.code LIKE "F02P%" OR ipc.code LIKE "B60D%" OR
    ipc.code LIKE "B60L%" OR ipc.code LIKE "B60S%" OR ipc.code LIKE "F01M%" OR ipc.code LIKE "F02F%" OR ipc.code LIKE "F16J%" OR
    ipc.code LIKE "B60G%" OR ipc.code LIKE "B60N%" OR ipc.code LIKE "B60T%" OR ipc.code LIKE "F01N%" OR ipc.code LIKE "F02G%" OR
    ipc.code LIKE "G01P%" OR ipc.code LIKE "B60H%" OR ipc.code LIKE "B60P%" OR ipc.code LIKE "B62D%" OR ipc.code LIKE "F01P%" OR
    ipc.code LIKE "F02M%" OR ipc.code LIKE "G05D%" OR ipc.code LIKE "B60J%" OR ipc.code LIKE "B60Q%" OR ipc.code LIKE "E01H%" OR
    ipc.code LIKE "F02B%" OR ipc.code LIKE "F02N%" OR ipc.code LIKE "G05G%" OR ipc.code LIKE "B61B%" OR ipc.code LIKE "B61H%" OR
    ipc.code LIKE "B62H%" OR ipc.code LIKE "B61C%" OR ipc.code LIKE "B61J%" OR ipc.code LIKE "B62J%" OR ipc.code LIKE "B61D%" OR
    ipc.code LIKE "B61K%" OR ipc.code LIKE "B62K%" OR ipc.code LIKE "B61F%" OR ipc.code LIKE "B61L%" OR ipc.code LIKE "B62M%" OR
    ipc.code LIKE "B61G%" OR ipc.code LIKE "B62B%" OR ipc.code LIKE "B65G%"
GROUP BY
    publication_number
  HAVING
    COUNT(ipc.code) > 0
),
assignee_data AS (
  SELECT
    publication_number,
    STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(assignee_harmonized) AS assignee_harmonized
  GROUP BY
    publication_number
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.ipc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  ipc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008
LIMIT 1000;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
df




,publication_number,publication_year,inventor_countries,ipc_codes,assignee_countries
0,US-2005042244-A1,2005,JP | JP,A01N49/00,US
1,US-2004169436-A1,2004,JP | JP | JP,H02N2/06 | F02M51/06 | H01L41/04 | F02D41/20 |...,JP
2,US-2005259275-A1,2005,JP,H04N1/393 | H04N1/21 | G06T3/40,JP
3,US-2005039187-A1,2005,US | US | US | US | US | US,G06F3/00,None
4,US-2001026407-A1,2001,JP | JP | JP | JP,G02B7/02 | G03B11/04 | G03B17/02 | G02B7/08,None
...,...,...,...,...,...
995,US-4797669-A,1989,US | US | US,H04Q9/00 | G08C15/06 | G08C15/00 | G08C19/02,US
996,US-5117691-A,1992,US,G01F1/698 | G01N27/18,US
997,US-5466371-A,1995,US | US,G01N33/48 | G01N33/483 | B04B5/02 | B04B7/16 |...,US
998,US-5330003-A,1994,US,E21B43/04 | E21B34/12 | E21B43/10 | E21B43/08,


Check if the data are retrieved correctly

In [ ]:
# SQL query to look at the names and countries of inventors for the patent with publication_number "US-5928469-A"
query = """
SELECT
  publication_number,
  inventor_harmonized.name AS inventor_name,
  inventor_harmonized.country_code AS inventor_country
FROM
  `patents-public-data.patents.publications_202310`,
  UNNEST(inventor_harmonized) AS inventor_harmonized
WHERE
  publication_number = 'US-2002075467-A1';
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df2 = results.to_dataframe()

# Display the DataFrame
df2

,publication_number,inventor_name,inventor_country
0,US-2002075467-A1,TANAKA KEIICHI,JP
1,US-2002075467-A1,BINNARD MIKE,US


Retrive the actual data without limiting the retreival to 1000

In [ ]:
# SQL query to only retrieve data
query = """
WITH inventor_data AS (
  SELECT
    publication_number,
    STRING_AGG(inventor_harmonized.country_code, ' | ') AS inventor_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(inventor_harmonized) AS inventor_harmonized
  WHERE
    inventor_harmonized.country_code IN ("AT", "AU", "BE", "CA", "DK", "FI", "DE", "IE", "JP", "NL", "NO", "SE", "CH", "GB", "US")
  GROUP BY
    publication_number
  HAVING
    COUNT(inventor_harmonized.country_code) > 0
),
ipc_data AS (
  SELECT
    publication_number,
    STRING_AGG(DISTINCT ipc.code, ' | ') AS ipc_codes
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(ipc) AS ipc
  WHERE
    ipc.code LIKE "B64B%" OR ipc.code LIKE "B64C%" OR ipc.code LIKE "B64D%" OR ipc.code LIKE "B64F%" OR ipc.code LIKE "B64G%" OR
    ipc.code LIKE "A61K%" OR ipc.code LIKE "C07K%" OR ipc.code LIKE "A61P%" OR ipc.code LIKE "C12N%" OR ipc.code LIKE "C07D%" OR
    ipc.code LIKE "C12P%" OR ipc.code LIKE "C07H%" OR ipc.code LIKE "C12Q%" OR ipc.code LIKE "C07J%" OR ipc.code LIKE "B41J%" OR
    ipc.code LIKE "G05F%" OR ipc.code LIKE "G06G%" OR ipc.code LIKE "G06T%" OR ipc.code LIKE "G07G%" OR ipc.code LIKE "H03K%" OR
    ipc.code LIKE "B41K%" OR ipc.code LIKE "G06C%" OR ipc.code LIKE "G06J%" OR ipc.code LIKE "G07B%" OR ipc.code LIKE "G09D%" OR
    ipc.code LIKE "H03L%" OR ipc.code LIKE "B43M%" OR ipc.code LIKE "G06D%" OR ipc.code LIKE "G06K%" OR ipc.code LIKE "G07C%" OR
    ipc.code LIKE "G09G%" OR ipc.code LIKE "G02F%" OR ipc.code LIKE "G06E%" OR ipc.code LIKE "G06M%" OR ipc.code LIKE "G07D%" OR
    ipc.code LIKE "G10L%" OR ipc.code LIKE "G03G%" OR ipc.code LIKE "G06F%" OR ipc.code LIKE "G06N%" OR ipc.code LIKE "G07F%" OR
    ipc.code LIKE "G11B%" OR ipc.code LIKE "B81B%" OR ipc.code LIKE "G11C%" OR ipc.code LIKE "H01L%" OR ipc.code LIKE "H03B%" OR
    ipc.code LIKE "H03H%" OR ipc.code LIKE "H04J%" OR ipc.code LIKE "H04Q%" OR ipc.code LIKE "B81C%" OR ipc.code LIKE "H01C%" OR
    ipc.code LIKE "H01P%" OR ipc.code LIKE "H03C%" OR ipc.code LIKE "H03J%" OR ipc.code LIKE "H04K%" OR ipc.code LIKE "H04R%" OR
    ipc.code LIKE "G03H%" OR ipc.code LIKE "H01F%" OR ipc.code LIKE "H01Q%" OR ipc.code LIKE "H03D%" OR ipc.code LIKE "H03M%" OR
    ipc.code LIKE "H04L%" OR ipc.code LIKE "H04S%" OR ipc.code LIKE "G09B%" OR ipc.code LIKE "H01G%" OR ipc.code LIKE "H01S%" OR
    ipc.code LIKE "H03F%" OR ipc.code LIKE "H04B%" OR ipc.code LIKE "H04M%" OR ipc.code LIKE "H05K%" OR ipc.code LIKE "G09C%" OR
    ipc.code LIKE "H01J%" OR ipc.code LIKE "H02J%" OR ipc.code LIKE "H03G%" OR ipc.code LIKE "H04H%" OR ipc.code LIKE "H04N%" OR
    ipc.code LIKE "A61B%" OR ipc.code LIKE "A61J%" OR ipc.code LIKE "B04B%" OR ipc.code LIKE "G01F%" OR ipc.code LIKE "G01N%" OR
    ipc.code LIKE "G02C%" OR ipc.code LIKE "G04D%" OR ipc.code LIKE "G12B%" OR ipc.code LIKE "A61C%" OR ipc.code LIKE "A61L%" OR
    ipc.code LIKE "C12M%" OR ipc.code LIKE "G01H%" OR ipc.code LIKE "G01R%" OR ipc.code LIKE "G03B%" OR ipc.code LIKE "G04F%" OR
    ipc.code LIKE "G21G%" OR ipc.code LIKE "A61D%" OR ipc.code LIKE "A61M%" OR ipc.code LIKE "F15C%" OR ipc.code LIKE "G01J%" OR
    ipc.code LIKE "G01S%" OR ipc.code LIKE "G03D%" OR ipc.code LIKE "G04G%" OR ipc.code LIKE "G21K%" OR ipc.code LIKE "A61F%" OR
    ipc.code LIKE "A61N%" OR ipc.code LIKE "G01B%" OR ipc.code LIKE "G01K%" OR ipc.code LIKE "G01T%" OR ipc.code LIKE "G03F%" OR
    ipc.code LIKE "G05B%" OR ipc.code LIKE "H05G%" OR ipc.code LIKE "A61G%" OR ipc.code LIKE "A62B%" OR ipc.code LIKE "G01C%" OR
    ipc.code LIKE "G01L%" OR ipc.code LIKE "G01W%" OR ipc.code LIKE "G04B%" OR ipc.code LIKE "G08C%" OR ipc.code LIKE "A61H%" OR
    ipc.code LIKE "B01L%" OR ipc.code LIKE "G01D%" OR ipc.code LIKE "G01M%" OR ipc.code LIKE "G02B%" OR ipc.code LIKE "G04C%" OR
    ipc.code LIKE "G09F%" OR ipc.code LIKE "A01N%" OR ipc.code LIKE "C01D%" OR ipc.code LIKE "C06B%" OR ipc.code LIKE "C08F%" OR
    ipc.code LIKE "C09F%" OR ipc.code LIKE "C10K%" OR ipc.code LIKE "C25B%" OR ipc.code LIKE "G03C%" OR ipc.code LIKE "A62D%" OR
    ipc.code LIKE "C01F%" OR ipc.code LIKE "C06C%" OR ipc.code LIKE "C08G%" OR ipc.code LIKE "C09G%" OR ipc.code LIKE "C10M%" OR
    ipc.code LIKE "D01C%" OR ipc.code LIKE "G21F%" OR ipc.code LIKE "B01J%" OR ipc.code LIKE "C01G%" OR ipc.code LIKE "C06D%" OR
    ipc.code LIKE "C08H%" OR ipc.code LIKE "C09H%" OR ipc.code LIKE "C11B%" OR ipc.code LIKE "D01F%" OR ipc.code LIKE "B09B%" OR
    ipc.code LIKE "C02F%" OR ipc.code LIKE "C07B%" OR ipc.code LIKE "C08J%" OR ipc.code LIKE "C09J%" OR ipc.code LIKE "C11C%" OR
    ipc.code LIKE "D06L%" OR ipc.code LIKE "B09C%" OR ipc.code LIKE "C05B%" OR ipc.code LIKE "C07C%" OR ipc.code LIKE "C08K%" OR
    ipc.code LIKE "C09K%" OR ipc.code LIKE "C11D%" OR ipc.code LIKE "F17C%" OR ipc.code LIKE "B27K%" OR ipc.code LIKE "C05C%" OR
    ipc.code LIKE "C07F%" OR ipc.code LIKE "C08L%" OR ipc.code LIKE "C10B%" OR ipc.code LIKE "C12S%" OR ipc.code LIKE "F17D%" OR
    ipc.code LIKE "B29B%" OR ipc.code LIKE "C05D%" OR ipc.code LIKE "C07G%" OR ipc.code LIKE "C09B%" OR ipc.code LIKE "C10C%" OR
    ipc.code LIKE "C14C%" OR ipc.code LIKE "F25J%" OR ipc.code LIKE "C01B%" OR ipc.code LIKE "C05F%" OR ipc.code LIKE "C08B%" OR
    ipc.code LIKE "C08C%" OR ipc.code LIKE "C09D%" OR ipc.code LIKE "C10J%" OR ipc.code LIKE "C23G%" OR ipc.code LIKE "F42D%" OR
    ipc.code LIKE "B60M%" OR ipc.code LIKE "F21M%" OR ipc.code LIKE "G08B%" OR ipc.code LIKE "H01H%" OR ipc.code LIKE "H02B%" OR
    ipc.code LIKE "H02P%" OR ipc.code LIKE "B61L%" OR ipc.code LIKE "F21P%" OR ipc.code LIKE "G08G%" OR ipc.code LIKE "H01K%" OR
    ipc.code LIKE "H02H%" OR ipc.code LIKE "H05C%" OR ipc.code LIKE "F21H%" OR ipc.code LIKE "F21Q%" OR ipc.code LIKE "G10K%" OR
    ipc.code LIKE "H01M%" OR ipc.code LIKE "H02K%" OR ipc.code LIKE "F21K%" OR ipc.code LIKE "F21S%" OR ipc.code LIKE "G21C%" OR
    ipc.code LIKE "H01R%" OR ipc.code LIKE "H02M%" OR ipc.code LIKE "F21L%" OR ipc.code LIKE "F21V%" OR ipc.code LIKE "G21D%" OR
    ipc.code LIKE "H01T%" OR ipc.code LIKE "H02N%" OR ipc.code LIKE "A01B%" OR ipc.code LIKE "B01D%" OR ipc.code LIKE "B23D%" OR
    ipc.code LIKE "B31F%" OR ipc.code LIKE "B68F%" OR ipc.code LIKE "D06B%" OR ipc.code LIKE "F04B%" OR ipc.code LIKE "F23N%" OR
    ipc.code LIKE "F41J%" OR ipc.code LIKE "A01C%" OR ipc.code LIKE "B01F%" OR ipc.code LIKE "B23F%" OR ipc.code LIKE "B41B%" OR
    ipc.code LIKE "C10F%" OR ipc.code LIKE "D06F%" OR ipc.code LIKE "F04C%" OR ipc.code LIKE "F23R%" OR ipc.code LIKE "F42C%" OR
    ipc.code LIKE "A01D%" OR ipc.code LIKE "B02B%" OR ipc.code LIKE "B23G%" OR ipc.code LIKE "B41C%" OR ipc.code LIKE "C12L%" OR
    ipc.code LIKE "D06G%" OR ipc.code LIKE "F04D%" OR ipc.code LIKE "F24B%" OR ipc.code LIKE "G01G%" OR ipc.code LIKE "A01F%" OR
    ipc.code LIKE "B02C%" OR ipc.code LIKE "B23H%" OR ipc.code LIKE "B41D%" OR ipc.code LIKE "C13C%" OR ipc.code LIKE "D06H%" OR
    ipc.code LIKE "F04F%" OR ipc.code LIKE "F24C%" OR ipc.code LIKE "G21J%" OR ipc.code LIKE "A01G%" OR ipc.code LIKE "B03B%" OR
    ipc.code LIKE "B23K%" OR ipc.code LIKE "B41F%" OR ipc.code LIKE "C13D%" OR ipc.code LIKE "D21B%" OR ipc.code LIKE "F15B%" OR
    ipc.code LIKE "F24D%" OR ipc.code LIKE "H05B%" OR ipc.code LIKE "A01J%" OR ipc.code LIKE "B03C%" OR ipc.code LIKE "B23P%" OR
    ipc.code LIKE "B41G%" OR ipc.code LIKE "C13G%" OR ipc.code LIKE "D21D%" OR ipc.code LIKE "F16C%" OR ipc.code LIKE "F24F%" OR
    ipc.code LIKE "H05F%" OR ipc.code LIKE "A01K%" OR ipc.code LIKE "B03D%" OR ipc.code LIKE "B23Q%" OR ipc.code LIKE "B41L%" OR
    ipc.code LIKE "C13H%" OR ipc.code LIKE "D21F%" OR ipc.code LIKE "F16D%" OR ipc.code LIKE "F24H%" OR ipc.code LIKE "H05H%" OR
    ipc.code LIKE "A01M%" OR ipc.code LIKE "B04C%" OR ipc.code LIKE "B24B%" OR ipc.code LIKE "B41N%" OR ipc.code LIKE "C14B%" OR
    ipc.code LIKE "D21G%" OR ipc.code LIKE "F16F%" OR ipc.code LIKE "F25B%" OR ipc.code LIKE "A21B%" OR ipc.code LIKE "B05B%" OR
    ipc.code LIKE "B24C%" OR ipc.code LIKE "B42B%" OR ipc.code LIKE "C23C%" OR ipc.code LIKE "E01C%" OR ipc.code LIKE "F16G%" OR
    ipc.code LIKE "F25C%" OR ipc.code LIKE "A21C%" OR ipc.code LIKE "B05C%" OR ipc.code LIKE "B25D%" OR ipc.code LIKE "B42C%" OR
    ipc.code LIKE "D01B%" OR ipc.code LIKE "E02D%" OR ipc.code LIKE "F16H%" OR ipc.code LIKE "F25D%" OR ipc.code LIKE "A22B%" OR
    ipc.code LIKE "B05D%" OR ipc.code LIKE "B25J%" OR ipc.code LIKE "B44B%" OR ipc.code LIKE "D01D%" OR ipc.code LIKE "E02F%" OR
    ipc.code LIKE "F16K%" OR ipc.code LIKE "F26B%" OR ipc.code LIKE "A22C%" OR ipc.code LIKE "B06B%" OR ipc.code LIKE "B26D%" OR
    ipc.code LIKE "B61B%" OR ipc.code LIKE "D01G%" OR ipc.code LIKE "E06C%" OR ipc.code LIKE "F16M%" OR ipc.code LIKE "F27B%" OR
    ipc.code LIKE "A23N%" OR ipc.code LIKE "B07B%" OR ipc.code LIKE "B26F%" OR ipc.code LIKE "B63G%" OR ipc.code LIKE "D01H%" OR
    ipc.code LIKE "E21B%" OR ipc.code LIKE "F16N%" OR ipc.code LIKE "F28B%" OR ipc.code LIKE "A24C%" OR ipc.code LIKE "B07C%" OR
    ipc.code LIKE "B27B%" OR ipc.code LIKE "B65B%" OR ipc.code LIKE "D02G%" OR ipc.code LIKE "E21C%" OR ipc.code LIKE "F22D%" OR
    ipc.code LIKE "F28C%" OR ipc.code LIKE "A41H%" OR ipc.code LIKE "B08B%" OR ipc.code LIKE "B27C%" OR ipc.code LIKE "B65C%" OR
    ipc.code LIKE "D02H%" OR ipc.code LIKE "E21D%" OR ipc.code LIKE "F23B%" OR ipc.code LIKE "F28D%" OR ipc.code LIKE "A42C%" OR
    ipc.code LIKE "B21B%" OR ipc.code LIKE "B27F%" OR ipc.code LIKE "B65G%" OR ipc.code LIKE "D02J%" OR ipc.code LIKE "E21F%" OR
    ipc.code LIKE "F23C%" OR ipc.code LIKE "F28F%" OR ipc.code LIKE "A43D%" OR ipc.code LIKE "B21D%" OR ipc.code LIKE "B27J%" OR
    ipc.code LIKE "B65H%" OR ipc.code LIKE "D03C%" OR ipc.code LIKE "F01B%" OR ipc.code LIKE "F23D%" OR ipc.code LIKE "F28G%" OR
    ipc.code LIKE "A45D%" OR ipc.code LIKE "B21F%" OR ipc.code LIKE "B27L%" OR ipc.code LIKE "B66B%" OR ipc.code LIKE "D03D%" OR
    ipc.code LIKE "F01C%" OR ipc.code LIKE "F23G%" OR ipc.code LIKE "F41A%" OR ipc.code LIKE "A47G%" OR ipc.code LIKE "B21H%" OR
    ipc.code LIKE "B28D%" OR ipc.code LIKE "B66C%" OR ipc.code LIKE "D03J%" OR ipc.code LIKE "F01D%" OR ipc.code LIKE "F23H%" OR
    ipc.code LIKE "F41B%" OR ipc.code LIKE "A47J%" OR ipc.code LIKE "B21J%" OR ipc.code LIKE "B30B%" OR ipc.code LIKE "B66D%" OR
    ipc.code LIKE "D04B%" OR ipc.code LIKE "F03B%" OR ipc.code LIKE "F23J%" OR ipc.code LIKE "F41C%" OR ipc.code LIKE "A47L%" OR
    ipc.code LIKE "B22C%" OR ipc.code LIKE "B31B%" OR ipc.code LIKE "B66F%" OR ipc.code LIKE "D04C%" OR ipc.code LIKE "F03C%" OR
    ipc.code LIKE "F23K%" OR ipc.code LIKE "F41F%" OR ipc.code LIKE "A62C%" OR ipc.code LIKE "B23B%" OR ipc.code LIKE "B31C%" OR
    ipc.code LIKE "B67B%" OR ipc.code LIKE "D05B%" OR ipc.code LIKE "F03D%" OR ipc.code LIKE "F23L%" OR ipc.code LIKE "F41G%" OR
    ipc.code LIKE "B01B%" OR ipc.code LIKE "B23C%" OR ipc.code LIKE "B31D%" OR ipc.code LIKE "B67C%" OR ipc.code LIKE "D05C%" OR
    ipc.code LIKE "F03G%" OR ipc.code LIKE "F23M%" OR ipc.code LIKE "F41H%" OR ipc.code LIKE "B60B%" OR ipc.code LIKE "B60K%" OR
    ipc.code LIKE "B60R%" OR ipc.code LIKE "F01L%" OR ipc.code LIKE "F02D%" OR ipc.code LIKE "F02P%" OR ipc.code LIKE "B60D%" OR
    ipc.code LIKE "B60L%" OR ipc.code LIKE "B60S%" OR ipc.code LIKE "F01M%" OR ipc.code LIKE "F02F%" OR ipc.code LIKE "F16J%" OR
    ipc.code LIKE "B60G%" OR ipc.code LIKE "B60N%" OR ipc.code LIKE "B60T%" OR ipc.code LIKE "F01N%" OR ipc.code LIKE "F02G%" OR
    ipc.code LIKE "G01P%" OR ipc.code LIKE "B60H%" OR ipc.code LIKE "B60P%" OR ipc.code LIKE "B62D%" OR ipc.code LIKE "F01P%" OR
    ipc.code LIKE "F02M%" OR ipc.code LIKE "G05D%" OR ipc.code LIKE "B60J%" OR ipc.code LIKE "B60Q%" OR ipc.code LIKE "E01H%" OR
    ipc.code LIKE "F02B%" OR ipc.code LIKE "F02N%" OR ipc.code LIKE "G05G%" OR ipc.code LIKE "B61B%" OR ipc.code LIKE "B61H%" OR
    ipc.code LIKE "B62H%" OR ipc.code LIKE "B61C%" OR ipc.code LIKE "B61J%" OR ipc.code LIKE "B62J%" OR ipc.code LIKE "B61D%" OR
    ipc.code LIKE "B61K%" OR ipc.code LIKE "B62K%" OR ipc.code LIKE "B61F%" OR ipc.code LIKE "B61L%" OR ipc.code LIKE "B62M%" OR
    ipc.code LIKE "B61G%" OR ipc.code LIKE "B62B%" OR ipc.code LIKE "B65G%"
GROUP BY
    publication_number
  HAVING
    COUNT(ipc.code) > 0
),
assignee_data AS (
  SELECT
    publication_number,
    STRING_AGG(assignee_harmonized.country_code, ' | ') AS assignee_countries
  FROM
    `patents-public-data.patents.publications_202310`,
    UNNEST(assignee_harmonized) AS assignee_harmonized
  GROUP BY
    publication_number
)
SELECT
  p.publication_number,
  CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) AS publication_year,
  i.inventor_countries,
  c.ipc_codes,
  a.assignee_countries
FROM
  `patents-public-data.patents.publications_202310` AS p
JOIN
  inventor_data AS i
ON
  p.publication_number = i.publication_number
JOIN
  ipc_data AS c
ON
  p.publication_number = c.publication_number
LEFT JOIN
  assignee_data AS a
ON
  p.publication_number = a.publication_number
WHERE
  LEFT(p.publication_number, 2) = 'US'
  AND CAST(SUBSTR(CAST(p.publication_date AS STRING), 1, 4) AS INT64) BETWEEN 1989 AND 2008;
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert to DataFrame
df = results.to_dataframe()

# Display the DataFrame
df


,publication_number,publication_year,inventor_countries,ipc_codes,assignee_countries
0,US-4887689-A,1989,JP,B60K17/348 | B60K23/08,JP
1,US-4832263-A,1989,US,B65H75/34 | A01G25/09,US
2,US-4887156-A,1989,JP,H04N19/51 | H04N19/137 | H04N19/91 | H04N19/11...,JP
3,US-4829779-A,1989,US | US,F25D29/00 | H04Q9/00 | F24F11/00 | F24F11/02 |...,US
4,US-4827776-A,1989,US | US,G01N1/08 | G01N33/04,US | US
...,...,...,...,...,...
3663438,US-2008216828-A1,2008,US | US | US | US | US,A61M16/10,US
3663439,US-2008280187-A1,2008,JP | JP | JP | JP,H01M8/04,JP
3663440,US-2008161182-A1,2008,US | US,B01J21/06 | B01J23/44 | B01J21/08 | B01J21/18 ...,|
3663441,US-7400132-B2,2008,JP,G01R33/07 | H03K3/59 | G01R33/06 | H03K17/97,JP


Save the retrieved data into a csv file

In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregate_patents_full.csv'
df.to_csv(output_file_path, index=False)

In [ ]:
# Function to handle None values during aggregation
def join_with_pipe(x):
    return ' | '.join([str(i) for i in x if i is not None])

# Grouping and aggregating the DataFrame, including assignee_countries
grouped_df = df.groupby(['publication_year', 'inventor_countries', 'publication_number']).agg({
    'ipc_codes': join_with_pipe,
    'assignee_countries': join_with_pipe  # Join assignee_countries with ' | ' separator
}).reset_index()

# Rename the columns
grouped_df.rename(columns={
    'publication_number': 'publication_number',
    'inventor_countries': 'inventor_country',
    'assignee_countries': 'assignee_country',
    'ipc_codes': 'patents_ipcs'
}, inplace=True)

grouped_df

,publication_year,inventor_country,publication_number,patents_ipcs,assignee_country
0,1989,AT,US-4794964-A,B27B29/00 | B27B7/02 | B27B1/00 | B27B5/18 | B...,AT
1,1989,AT,US-4795474-A,F16C11/06 | F16C11/04 | A61F2/60 | A61F2/50 | ...,DE
2,1989,AT,US-4795646-A,A61K9/16 | A61K31/17 | A61P9/00,AT
3,1989,AT,US-4795750-A,A61P9/12 | C07D403/12 | A61K31/505 | C07D401/1...,AT
4,1989,AT,US-4795967-A,G01F23/26,AT
...,...,...,...,...,...
3663438,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G7/002 | A61G7/015 | A61G7/005 | A61G7/057 ...,| | | | | | | | | | | | | | | ...
3663439,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G7/057 | A61G7/00 | B60B33/02 | A61G7/002 |...,| | | | | | | | | | | | | | | ...
3663440,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G7/018 | A61G7/015 | A61G7/00 | A61G7/053 |...,| | | | | | | | | | | | | | | ...
3663441,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G7/008 | A61G12/00 | A61G7/005 | A61G7/057 ...,US


In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df.shape[0]}")
print(f"Grouped DataFrame rows: {grouped_df.shape[0]}")

Original DataFrame rows: 3663443
Grouped DataFrame rows: 3663443


Save the dataset

In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregate_patents_ipcs_with_number_in_one_row.csv'
grouped_df.to_csv(output_file_path, index=False)

Remove the numbers after the first four characters of the "patents_ipcs".


Step 1: Define a Function to Simplify the ipc Codes

1. Function Definition:

def simplify_ipc_codes(codes): Defines a function named simplify_ipc_codes that takes a single argument codes.

2. Initialize List:

simplified_codes = []: Initializes an empty list to store the simplified codes.

3. Split Codes:

for code in codes.split('|'): Iterates over each code in the codes string, splitting the string by the | separator.

4. Regular Expression Match:

simplified_code = re.match(r'^[A-Z]\d{2}[A-Z]', code): Uses a regular expression to match the pattern where a code starts with an uppercase letter, followed by two digits, and another uppercase letter. This is intended to capture a simplified form of the ipc code.

5. Conditional Append:

if simplified_code: Checks if the regular expression found a match.
simplified_codes.append(simplified_code.group(0)): If a match is found, appends the simplified code to the simplified_codes list.
else: simplified_codes.append(code): If no match is found, appends the original code to the simplified_codes list.

6. Join Simplified Codes:

return '|'.join(simplified_codes): Joins the simplified codes with a | separator and returns the resulting string.



Step 2: Define a Function to Simplify Codes to Prefixes

1. Function Definition:

def simplify_to_prefix(codes): Defines a function named simplify_to_prefix that takes a single argument codes.

2. Simplify Codes to Prefixes:

simplified_codes = [code[:4] for code in codes.split('|')]: Uses a list comprehension to iterate over each code in the codes string, splitting the string by the | separator, and then taking only the first four characters of each code.

3. Join Simplified Codes:

return '|'.join(simplified_codes): Joins the simplified codes with a | separator and returns the resulting string.


Step 3: Apply the Function to the 'patents_ipcs' Column

1. Apply Function:

df['patents_ipcs'] = df['patents_ipcs'].apply(simplify_to_prefix): Applies the simplify_to_prefix function to each value in the patents_ipcs column of the DataFrame df.

2. Print DataFrame:

print(df.head()): Prints the first few rows of the DataFrame df to verify the changes.

In [ ]:
# Define a function to simplify the ipc codes
def simplify_ipc_codes(codes):
    simplified_codes = []
    for code in codes.split('|'):
        simplified_code = re.match(r'^[A-Z]\d{2}[A-Z]', code)
        if simplified_code:
            simplified_codes.append(simplified_code.group(0))
        else:
            simplified_codes.append(code)
    return '|'.join(simplified_codes)

# Adjust the function to remove characters after the first four and ensure at least 4 characters long
def simplify_to_prefix(codes):
    simplified_codes = []
    for code in codes.split('|'):
        code = code.strip()
        if len(code) < 4:
            simplified_code = code.ljust(4)  # Pad with spaces if less than 4 characters
        else:
            simplified_code = code[:4]  # Take only the first 4 characters
        simplified_codes.append(simplified_code)
    return '|'.join(simplified_codes)

# Apply the function to the 'patents_ipcs' column
grouped_df['patents_ipcs'] = grouped_df['patents_ipcs'].apply(simplify_to_prefix)
grouped_df
df = grouped_df

In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df.shape[0]}")

Original DataFrame rows: 3663443


In [ ]:
df

,publication_year,inventor_country,publication_number,patents_ipcs,assignee_country
0,1989,AT,US-4794964-A,B27B|B27B|B27B|B27B|B23D,AT
1,1989,AT,US-4795474-A,F16C|F16C|A61F|A61F|A61F|A61F|A61F,DE
2,1989,AT,US-4795646-A,A61K|A61K|A61P,AT
3,1989,AT,US-4795750-A,A61P|C07D|A61K|C07D|C07D|A61K,AT
4,1989,AT,US-4795967-A,G01F,AT
...,...,...,...,...,...
3663438,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G|A61G|A61G|A61G|A61G|A61G|B60B|G01G|A61G|A...,| | | | | | | | | | | | | | | ...
3663439,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G|A61G|B60B|A61G|B60B|A61G|G01G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3663440,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61H,| | | | | | | | | | | | | | | ...
3663441,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G|A61G|A61G|A61G|A61G|G01G|A61G|B60B|A61G|A...,US


Sorting the DataFrame by 'publication_year' and 'inventor_country'

1. sort_values Method:

df.sort_values(by=['publication_year', 'inventor_country'])
This method is used to sort the DataFrame df based on the values in the specified columns.

2. by Parameter:

by=['publication_year', 'inventor_country']
This specifies the columns to sort by.
The DataFrame will first be sorted by the values in the publication_year column.
Within each publication_year, the DataFrame will be further sorted by the values in the inventor_country column.

3. Return Value:

The sort_values method returns a new DataFrame that is sorted according to the specified columns.
The sorted DataFrame is assigned to the variable df_sorted.


The purpose of this code is to sort the DataFrame df in an orderly manner based on the values in the publication_year and inventor_country columns. Sorting the DataFrame can make it easier to analyze trends and patterns in the data, especially when dealing with time series data or when trying to compare records within specific categories.

In [ ]:
df_sorted = df.sort_values(by=['publication_year', 'inventor_country'])

# Display the sorted DataFrame
print(df_sorted.head())
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregate_patents_ipcs_final.csv'
df_sorted.to_csv(output_file_path, index=False)

   publication_year inventor_country publication_number  \
0              1989               AT       US-4794964-A   
1              1989               AT       US-4795474-A   
2              1989               AT       US-4795646-A   
3              1989               AT       US-4795750-A   
4              1989               AT       US-4795967-A   

                         patents_ipcs assignee_country  
0            B27B|B27B|B27B|B27B|B23D               AT  
1  F16C|F16C|A61F|A61F|A61F|A61F|A61F               DE  
2                      A61K|A61K|A61P               AT  
3       A61P|C07D|A61K|C07D|C07D|A61K               AT  
4                                G01F               AT  


In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {df_sorted.shape[0]}")

Original DataFrame rows: 3663443


Check if there are multiple codes for each country in the same year, to make sure that the codes are properly executed

In [ ]:
# Filter the DataFrame for the US in the year 2005
filtered_df = df_sorted[(df_sorted['publication_year'] == 2005) & (df_sorted['inventor_country'] == 'US')]

# Count the occurrences of "B65B" in the 'patents_ipcs' column
count_a61p = filtered_df['patents_ipcs'].str.split('|').apply(lambda x: x.count('A61P')).sum()

count_a61p

3413

Load the datasets here: Optional

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregate_patents_ipcs_final.csv')
df


,publication_year,inventor_country,publication_number,patents_ipcs,assignee_country
0,1989,AT,US-4794964-A,B27B|B27B|B27B|B27B|B23D,AT
1,1989,AT,US-4795474-A,F16C|F16C|A61F|A61F|A61F|A61F|A61F,DE
2,1989,AT,US-4795646-A,A61K|A61K|A61P,AT
3,1989,AT,US-4795750-A,A61P|C07D|A61K|C07D|C07D|A61K,AT
4,1989,AT,US-4795967-A,G01F,AT
...,...,...,...,...,...
3663438,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201847-A1,A61G|A61G|A61G|A61G|A61G|A61G|B60B|G01G|A61G|A...,| | | | | | | | | | | | | | | ...
3663439,2008,US | US | US | US | US | US | US | US | US | U...,US-2008201851-A1,A61G|A61G|B60B|A61G|B60B|A61G|G01G|A61G|A61G|A...,| | | | | | | | | | | | | | | ...
3663440,2008,US | US | US | US | US | US | US | US | US | U...,US-2008289108-A1,A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61G|A61H,| | | | | | | | | | | | | | | ...
3663441,2008,US | US | US | US | US | US | US | US | US | U...,US-7406731-B2,A61G|A61G|A61G|A61G|A61G|G01G|A61G|B60B|A61G|A...,US


Corrected Final Codes for calculating frational counts and modes for inventor_country and assignee_country columns

In [ ]:
# Function to aggregate unique names and country codes into lists
def aggregate_unique(series):
    return '|'.join(series.dropna().unique())

# Function to calculate fractional shares from pipe-separated strings
def calculate_fraction(countries_str):
    if not countries_str:
        return {}
    countries = [country.strip() for country in countries_str.split('|')]  # Remove leading/trailing spaces
    total = len(countries)
    return '|'.join([f'{country}:{countries.count(country) / total:.4f}' for country in set(countries)])

# Function to find the mode of country codes
def find_mode(countries_str):
    if not countries_str:
        return None
    countries = pd.Series([country.strip() for country in countries_str.split('|')])
    mode_series = countries.mode()
    return mode_series.tolist() if not mode_series.empty else None

# Aggregate data by publication_number
aggregated_data = df.groupby('publication_number').agg({
    'inventor_country': aggregate_unique,
    'assignee_country': aggregate_unique,
    'publication_year': 'first',  # Retain the first occurrence
    'patents_ipcs': 'first'       # Retain the first occurrence
}).reset_index()

# Apply functions to calculate fractional shares and modes
aggregated_data['inventors_country_frac'] = aggregated_data['inventor_country'].apply(calculate_fraction)
aggregated_data['assignees_country_frac'] = aggregated_data['assignee_country'].apply(calculate_fraction)
aggregated_data['inventors_country_mode'] = aggregated_data['inventor_country'].apply(find_mode)
aggregated_data['assignees_country_mode'] = aggregated_data['assignee_country'].apply(find_mode)
aggregated_data

,publication_number,inventor_country,assignee_country,publication_year,patents_ipcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode
0,US-2001000001-A1,US | US,,2001,C11D|C11D|B08B|C23G|C11D|C23G|C11D|C11D|C11D|C...,US:1.0000,{},[US],None
1,US-2001000002-A1,DE | DE | DE | DE,,2001,G01B|D21G|G01B|G01B|B25J|G01B,DE:1.0000,{},[DE],None
2,US-2001000003-A1,US,,2001,B66B,US:1.0000,{},[US],None
3,US-2001000004-A1,US,,2001,B65G|G07F|G07F,US:1.0000,{},[US],None
4,US-2001000005-A1,US | US | US | US | US,,2001,H01L|H01L|H01L|C09K|H01L|H01L|H01L,US:1.0000,{},[US],None
...,...,...,...,...,...,...,...,...,...
3663438,US-RE40607-E,US | US,US,2008,A61B|G01N|H05G,US:1.0000,US:1.0000,[US],[US]
3663439,US-RE40608-E,GB | GB | GB,US,2008,A61B|A61B|G01S,GB:1.0000,US:1.0000,[GB],[US]
3663440,US-RE40609-E,JP,JP,2008,G02B|G03B|G02B|H04N|G03B|G03B|H04N|G02B,JP:1.0000,JP:1.0000,[JP],[JP]
3663441,US-RE40610-E,JP,JP,2008,G02B|G02B|H04N|H04N|G03B|G02B|G03B|G03B,JP:1.0000,JP:1.0000,[JP],[JP]


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final.csv'
aggregated_data.to_csv(output_file_path, index=False)

In [ ]:
# Display the number of rows in the original and grouped DataFrames
print(f"Original DataFrame rows: {aggregated_data.shape[0]}")

Original DataFrame rows: 3663443


Change the dataframe from aggregated_data to df

In [ ]:
df = aggregated_data

Load the created dataframe of aggregated_data into the name of df, as it is more efficient. The calculation of fractions and modes takes 54 minutes, so it is a good idea to just load it here for saving time.

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final.csv')
df

,publication_number,inventor_country,assignee_country,publication_year,patents_ipcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode
0,US-2001000001-A1,US | US,NaN,2001,C11D|C11D|B08B|C23G|C11D|C23G|C11D|C11D|C11D|C...,US:1.0000,{},['US'],NaN
1,US-2001000002-A1,DE | DE | DE | DE,NaN,2001,G01B|D21G|G01B|G01B|B25J|G01B,DE:1.0000,{},['DE'],NaN
2,US-2001000003-A1,US,NaN,2001,B66B,US:1.0000,{},['US'],NaN
3,US-2001000004-A1,US,NaN,2001,B65G|G07F|G07F,US:1.0000,{},['US'],NaN
4,US-2001000005-A1,US | US | US | US | US,NaN,2001,H01L|H01L|H01L|C09K|H01L|H01L|H01L,US:1.0000,{},['US'],NaN
...,...,...,...,...,...,...,...,...,...
3663438,US-RE40607-E,US | US,US,2008,A61B|G01N|H05G,US:1.0000,US:1.0000,['US'],['US']
3663439,US-RE40608-E,GB | GB | GB,US,2008,A61B|A61B|G01S,GB:1.0000,US:1.0000,['GB'],['US']
3663440,US-RE40609-E,JP,JP,2008,G02B|G03B|G02B|H04N|G03B|G03B|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP']
3663441,US-RE40610-E,JP,JP,2008,G02B|G02B|H04N|H04N|G03B|G02B|G03B|G03B,JP:1.0000,JP:1.0000,['JP'],['JP']


Generate a new column for fractional counts of ipc codes of patents

In [ ]:
# Function to calculate fractional shares from pipe-separated strings
def calculate_fraction(codes_str):
    if not codes_str:
        return {}
    codes = [code.strip() for code in codes_str.split('|')]  # Remove leading/trailing spaces
    total = len(codes)
    return '|'.join([f'{code}:{codes.count(code) / total:.4f}' for code in set(codes)])

# Apply the function to calculate fractional shares for the patents_ipcs column
df['patents_ipcs_frac'] = df['patents_ipcs'].apply(calculate_fraction)
df

,publication_number,inventor_country,assignee_country,publication_year,patents_ipcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_ipcs_frac
0,US-2001000001-A1,US | US,NaN,2001,C11D|C11D|B08B|C23G|C11D|C23G|C11D|C11D|C11D|C...,US:1.0000,{},['US'],NaN,C11D:0.5833|B08B:0.0833|C23G:0.3333
1,US-2001000002-A1,DE | DE | DE | DE,NaN,2001,G01B|D21G|G01B|G01B|B25J|G01B,DE:1.0000,{},['DE'],NaN,D21G:0.1667|B25J:0.1667|G01B:0.6667
2,US-2001000003-A1,US,NaN,2001,B66B,US:1.0000,{},['US'],NaN,B66B:1.0000
3,US-2001000004-A1,US,NaN,2001,B65G|G07F|G07F,US:1.0000,{},['US'],NaN,G07F:0.6667|B65G:0.3333
4,US-2001000005-A1,US | US | US | US | US,NaN,2001,H01L|H01L|H01L|C09K|H01L|H01L|H01L,US:1.0000,{},['US'],NaN,H01L:0.8571|C09K:0.1429
...,...,...,...,...,...,...,...,...,...,...
3663438,US-RE40607-E,US | US,US,2008,A61B|G01N|H05G,US:1.0000,US:1.0000,['US'],['US'],A61B:0.3333|G01N:0.3333|H05G:0.3333
3663439,US-RE40608-E,GB | GB | GB,US,2008,A61B|A61B|G01S,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333
3663440,US-RE40609-E,JP,JP,2008,G02B|G03B|G02B|H04N|G03B|G03B|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750
3663441,US-RE40610-E,JP,JP,2008,G02B|G02B|H04N|H04N|G03B|G02B|G03B|G03B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final_final.csv'
df.to_csv(output_file_path, index=False)

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final_final.csv')
df

,publication_number,inventor_country,assignee_country,publication_year,patents_ipcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_ipcs_frac
0,US-2001000001-A1,US | US,NaN,2001,C11D|C11D|B08B|C23G|C11D|C23G|C11D|C11D|C11D|C...,US:1.0000,{},['US'],NaN,C11D:0.5833|B08B:0.0833|C23G:0.3333
1,US-2001000002-A1,DE | DE | DE | DE,NaN,2001,G01B|D21G|G01B|G01B|B25J|G01B,DE:1.0000,{},['DE'],NaN,D21G:0.1667|B25J:0.1667|G01B:0.6667
2,US-2001000003-A1,US,NaN,2001,B66B,US:1.0000,{},['US'],NaN,B66B:1.0000
3,US-2001000004-A1,US,NaN,2001,B65G|G07F|G07F,US:1.0000,{},['US'],NaN,G07F:0.6667|B65G:0.3333
4,US-2001000005-A1,US | US | US | US | US,NaN,2001,H01L|H01L|H01L|C09K|H01L|H01L|H01L,US:1.0000,{},['US'],NaN,H01L:0.8571|C09K:0.1429
...,...,...,...,...,...,...,...,...,...,...
3663438,US-RE40607-E,US | US,US,2008,A61B|G01N|H05G,US:1.0000,US:1.0000,['US'],['US'],A61B:0.3333|G01N:0.3333|H05G:0.3333
3663439,US-RE40608-E,GB | GB | GB,US,2008,A61B|A61B|G01S,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333
3663440,US-RE40609-E,JP,JP,2008,G02B|G03B|G02B|H04N|G03B|G03B|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750
3663441,US-RE40610-E,JP,JP,2008,G02B|G02B|H04N|H04N|G03B|G02B|G03B|G03B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750


In [ ]:
# Load the mapping dataset
mapping_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/Patents_IPCs_Final.csv')
mapping_df

,Unnamed: 0,patents_ipcs,patents_cpcs,Consistency,ID,sector,sector_tech,tech_sech
0,1,B64B,B64B,True,1,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
1,2,B64C,B64C,True,2,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
2,3,B64D,B64D,True,3,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
3,4,B64F,B64F,True,4,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
4,5,B64G,B64G,True,5,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
...,...,...,...,...,...,...,...,...
444,445,B61L,B61L,True,445,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
445,446,B62M,B62M,True,446,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
446,447,B61G,B61G,True,447,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
447,448,B62B,B62B,True,448,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech


In [ ]:
# Create a dictionary to map ipc codes to sectors
ipc_to_sector = dict(zip(mapping_df['patents_ipcs'], mapping_df['sector_tech']))

# Function to map ipc codes to sectors
def map_ipcs_to_sectors(ipc_str):
    ipcs = ipc_str.split('|')
    sectors = [ipc_to_sector.get(ipc.strip(), 'Unknown') for ipc in ipcs]
    return '|'.join(sectors)

# Apply the function to the patents_ipcs column
df['sectors'] = df['patents_ipcs'].apply(map_ipcs_to_sectors)

# Function to calculate sector fractions based on ipc fractions and inventor fractions
def calculate_sector_fraction(row):
    ipc_fractions = row['patents_ipcs_frac'].split('|')
    inventor_fractions = row['inventors_country_frac'].split('|')

    sector_fractions = {}
    for ipc_frac in ipc_fractions:
        ipc_code, ipc_share = ipc_frac.split(':')
        ipc_share = float(ipc_share)

        sector = ipc_to_sector.get(ipc_code.strip(), 'Unknown')

        if sector not in sector_fractions:
            sector_fractions[sector] = 0

        for inv_frac in inventor_fractions:
            inv_country, inv_share = inv_frac.split(':')
            inv_share = float(inv_share)
            sector_fractions[sector] += ipc_share * inv_share

    return '|'.join([f'{sector}:{share:.4f}' for sector, share in sector_fractions.items()])

# Apply the function to calculate sector fractions
df['sector_fractions'] = df.apply(calculate_sector_fraction, axis=1)
df


,publication_number,inventor_country,assignee_country,publication_year,patents_ipcs,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,patents_ipcs_frac,sectors,sector_fractions
0,US-2001000001-A1,US | US,NaN,2001,C11D|C11D|B08B|C23G|C11D|C23G|C11D|C11D|C11D|C...,US:1.0000,{},['US'],NaN,C11D:0.5833|B08B:0.0833|C23G:0.3333,Chemicals_excluding_pharmaceuticals|Chemicals_...,Chemicals_excluding_pharmaceuticals:0.9166|Mac...
1,US-2001000002-A1,DE | DE | DE | DE,NaN,2001,G01B|D21G|G01B|G01B|B25J|G01B,DE:1.0000,{},['DE'],NaN,D21G:0.1667|B25J:0.1667|G01B:0.6667,Medical_precision_and_optical_instruments|Mach...,Machinery_and_equipment:0.3334|Medical_precisi...
2,US-2001000003-A1,US,NaN,2001,B66B,US:1.0000,{},['US'],NaN,B66B:1.0000,Machinery_and_equipment,Machinery_and_equipment:1.0000
3,US-2001000004-A1,US,NaN,2001,B65G|G07F|G07F,US:1.0000,{},['US'],NaN,G07F:0.6667|B65G:0.3333,Rail_and_transport_equipment|Office_accounting...,Office_accounting_and_computing_machinery:0.66...
4,US-2001000005-A1,US | US | US | US | US,NaN,2001,H01L|H01L|H01L|C09K|H01L|H01L|H01L,US:1.0000,{},['US'],NaN,H01L:0.8571|C09K:0.1429,Radio_TV_and_communications_equipment|Radio_TV...,Radio_TV_and_communications_equipment:0.8571|C...
...,...,...,...,...,...,...,...,...,...,...,...,...
3663438,US-RE40607-E,US | US,US,2008,A61B|G01N|H05G,US:1.0000,US:1.0000,['US'],['US'],A61B:0.3333|G01N:0.3333|H05G:0.3333,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:0.9999
3663439,US-RE40608-E,GB | GB | GB,US,2008,A61B|A61B|G01S,GB:1.0000,US:1.0000,['GB'],['US'],A61B:0.6667|G01S:0.3333,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:1.0000
3663440,US-RE40609-E,JP,JP,2008,G02B|G03B|G02B|H04N|G03B|G03B|H04N|G02B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:0.75...
3663441,US-RE40610-E,JP,JP,2008,G02B|G02B|H04N|H04N|G03B|G02B|G03B|G03B,JP:1.0000,JP:1.0000,['JP'],['JP'],G02B:0.3750|H04N:0.2500|G03B:0.3750,Medical_precision_and_optical_instruments|Medi...,Medical_precision_and_optical_instruments:0.75...


In [ ]:
# Extract sector and country fractions into a new DataFrame
sector_country_fractions = []

for idx, row in df.iterrows():
    sectors = row['sectors'].split('|')
    inventors_country_frac = row['inventors_country_frac'].split('|')
    for sector in set(sectors):
        for country_frac in inventors_country_frac:
            country, frac = country_frac.split(':')
            frac = float(frac)
            sector_country_fractions.append([
                row['publication_number'], row['assignee_country'], row['publication_year'], row['assignees_country_frac'],
                row['inventors_country_mode'], row['assignees_country_mode'], sector, country, frac
            ])

# Convert to DataFrame
scf_df = pd.DataFrame(sector_country_fractions, columns=[
    'publication_number', 'assignee_country', 'publication_year', 'assignees_country_frac',
    'inventors_country_mode', 'assignees_country_mode', 'sector', 'country', 'fraction'
])

scf_df

,publication_number,assignee_country,publication_year,assignees_country_frac,inventors_country_mode,assignees_country_mode,sector,country,fraction
0,US-2001000001-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0
1,US-2001000001-A1,NaN,2001,{},['US'],NaN,Machinery_and_equipment,US,1.0
2,US-2001000002-A1,NaN,2001,{},['DE'],NaN,Machinery_and_equipment,DE,1.0
3,US-2001000002-A1,NaN,2001,{},['DE'],NaN,Medical_precision_and_optical_instruments,DE,1.0
4,US-2001000003-A1,NaN,2001,{},['US'],NaN,Machinery_and_equipment,US,1.0
...,...,...,...,...,...,...,...,...,...
4972055,US-RE40609-E,JP,2008,JP:1.0000,['JP'],['JP'],Radio_TV_and_communications_equipment,JP,1.0
4972056,US-RE40609-E,JP,2008,JP:1.0000,['JP'],['JP'],Medical_precision_and_optical_instruments,JP,1.0
4972057,US-RE40610-E,JP,2008,JP:1.0000,['JP'],['JP'],Radio_TV_and_communications_equipment,JP,1.0
4972058,US-RE40610-E,JP,2008,JP:1.0000,['JP'],['JP'],Medical_precision_and_optical_instruments,JP,1.0


In [ ]:
# Calculate total fractions by sector and country
sector_totals = scf_df.groupby('sector')['fraction'].sum().reset_index()
sector_totals.columns = ['sector', 'total_sector_fraction']

country_totals = scf_df.groupby('country')['fraction'].sum().reset_index()
country_totals.columns = ['country', 'total_country_fraction']

total_fraction = scf_df['fraction'].sum()

# Merge totals with the sector_country_fractions DataFrame
scf_df = pd.merge(scf_df, sector_totals, on='sector')
scf_df = pd.merge(scf_df, country_totals, on='country')

# Calculate the RTA for each sector and country
scf_df['RTA'] = (scf_df['fraction'] / scf_df['total_sector_fraction']) / (scf_df['total_country_fraction'] / total_fraction)

# Normalize the RTA values
scf_df['Normalized_RTA'] = (scf_df['RTA'] - 1) / (scf_df['RTA'] + 1)

scf_df

,publication_number,assignee_country,publication_year,assignees_country_frac,inventors_country_mode,assignees_country_mode,sector,country,fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA
0,US-2001000001-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0,418981.9957,2.625760e+06,0.000004,-0.999991
1,US-2001000005-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0,418981.9957,2.625760e+06,0.000004,-0.999991
2,US-2001000037-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0,418981.9957,2.625760e+06,0.000004,-0.999991
3,US-2001000038-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0,418981.9957,2.625760e+06,0.000004,-0.999991
4,US-2001000039-A1,NaN,2001,{},['US'],NaN,Chemicals_excluding_pharmaceuticals,US,1.0,418981.9957,2.625760e+06,0.000004,-0.999991
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972055,US-5085383-A,GB,1992,GB:1.0000,['IE'],['GB'],Aircraft_and_spacecraft,IE,1.0,17794.9998,5.559012e+03,0.048507,-0.907474
4972056,US-5417385-A,GB,1995,GB:1.0000,['IE'],['GB'],Aircraft_and_spacecraft,IE,1.0,17794.9998,5.559012e+03,0.048507,-0.907474
4972057,US-5609313-A,IE,1997,IE:1.0000,['IE'],['IE'],Aircraft_and_spacecraft,IE,1.0,17794.9998,5.559012e+03,0.048507,-0.907474
4972058,US-6606826-B2,NaN,2003,{},['IE'],NaN,Aircraft_and_spacecraft,IE,1.0,17794.9998,5.559012e+03,0.048507,-0.907474


In [ ]:
# Group by sector, year, and country and calculate aggregated RTA
aggregated_rta_df = scf_df.groupby(['sector', 'publication_year', 'country']).agg(
    total_fraction=('fraction', 'sum'),
    total_sector_fraction=('total_sector_fraction', 'first'),  # Assuming the same value for the entire group
    total_country_fraction=('total_country_fraction', 'first')  # Assuming the same value for the entire group
).reset_index()

# Calculate the RTA for each sector, year, and country
aggregated_rta_df['RTA'] = (aggregated_rta_df['total_fraction'] / aggregated_rta_df['total_sector_fraction']) / (aggregated_rta_df['total_country_fraction'] / total_fraction)

# Normalize the RTA values for each sector, year, and country
aggregated_rta_df['Normalized_RTA'] = (aggregated_rta_df['RTA'] - 1) / (aggregated_rta_df['RTA'] + 1)


aggregated_rta_df

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA
0,Aircraft_and_spacecraft,1989,AT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607
1,Aircraft_and_spacecraft,1989,AU,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738
2,Aircraft_and_spacecraft,1989,BE,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493
3,Aircraft_and_spacecraft,1989,CA,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956
4,Aircraft_and_spacecraft,1989,CH,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124
...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JP,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266
2945,Rail_and_transport_equipment,2008,NL,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201
2946,Rail_and_transport_equipment,2008,NO,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482
2947,Rail_and_transport_equipment,2008,SE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971


In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final_final_final.csv'
aggregated_rta_df.to_csv(output_file_path, index=False)

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/aggregated_data_final_final_final_final.csv')
df

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA
0,Aircraft_and_spacecraft,1989,AT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607
1,Aircraft_and_spacecraft,1989,AU,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738
2,Aircraft_and_spacecraft,1989,BE,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493
3,Aircraft_and_spacecraft,1989,CA,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956
4,Aircraft_and_spacecraft,1989,CH,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124
...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JP,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266
2945,Rail_and_transport_equipment,2008,NL,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201
2946,Rail_and_transport_equipment,2008,NO,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482
2947,Rail_and_transport_equipment,2008,SE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971


In [ ]:
# Mapping dictionary to unify country codes
mapping = {
    'AT': 'AUT',
    'AU': 'AUS',
    'BE': 'BEL',
    'CA': 'CAN',
    'CH': 'CHE',
    'DE': 'DEU',
    'DK': 'DNK',
    'FI': 'FIN',
    'GB': 'GBR',
    'IE': 'IRL',
    'JP': 'JPN',
    'NL': 'NLD',
    'NO': 'NOR',
    'SE': 'SWE',
    'US': 'USA'}
# Apply the mapping to the dataframe
df['country'] = df['country'].map(mapping)

# Print unique values of the 'country_code' column
unique_country_codes = df['country'].unique()
print(unique_country_codes)

['AUT' 'AUS' 'BEL' 'CAN' 'CHE' 'DEU' 'GBR' 'JPN' 'NLD' 'SWE' 'USA' 'NOR'
 'DNK' 'FIN' 'IRL']


In [ ]:
import requests
import pandas as pd
from io import StringIO

# Define the API URL
api_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.CAN+AUS+IRL+USA+GBR+SWE+FIN+DNK+NLD+CHE+NOR+BEL+AUT+DEU+JPN.B1GQ_R.USD_PPP.?startPeriod=1989&endPeriod=2008"

# Set the headers to request CSV format
headers = {
    "Accept": "application/vnd.sdmx.data+csv; charset=utf-8; version=2"
}

# Make the API request
response = requests.get(api_url, headers=headers)
response.raise_for_status()  # Ensure we raise an error for bad responses

# Decode the response content
data = response.content.decode('utf-8')

# Convert the CSV response into a pandas DataFrame
data_io = StringIO(data)
df_gdp = pd.read_csv(data_io, sep=',')

# Sort the dataframe by 'REF_AREA' and 'TIME_PERIOD' columns
df_gdp = df_gdp.sort_values(by=['REF_AREA', 'TIME_PERIOD'])

# Rename the columns
df_gdp.rename(columns={
    "REF_AREA": "country",
    "TIME_PERIOD": "publication_year",
    "OBS_VALUE": "Real_GDP_2015_Billion"
}, inplace=True)

# Keep only the specified columns
df_gdp = df_gdp[["country", "publication_year", "Real_GDP_2015_Billion"]]

# Save DataFrame to a CSV file
output_file = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/oecd_gdp_data.csv'
df_gdp.to_csv(output_file, index=False)

print(f"Data has been saved to {output_file}")

#Merge with the dataset
import pandas as pd

# Convert the 'Time' column to integers in both datasets for consistent merging
df_gdp['publication_year'] = df_gdp['publication_year'].astype(int)
df['publication_year'] = df['publication_year'].astype(int)

# Merge the datasets on 'Time' and 'COU'
merged_df = df.merge(df_gdp, on=['publication_year', 'country'], how='left')

# Display the first few rows of the merged DataFrame
print(merged_df.head())


Data has been saved to /content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/oecd_gdp_data.csv
                    sector  publication_year country  total_fraction  \
0  Aircraft_and_spacecraft              1989     AUT             2.0   
1  Aircraft_and_spacecraft              1989     AUS             3.0   
2  Aircraft_and_spacecraft              1989     BEL             1.0   
3  Aircraft_and_spacecraft              1989     CAN             1.0   
4  Aircraft_and_spacecraft              1989     CHE             1.0   

   total_sector_fraction  total_country_fraction       RTA  Normalized_RTA  \
0             17794.9998              16210.8844  0.033268       -0.935607   
1             17794.9998              36588.6751  0.022109       -0.956738   
2             17794.9998              22672.3793  0.011893       -0.976493   
3             17794.9998             106654.5921  0.002528       -0.994956   
4             17794.9998              49316.79

Import and Merge Internationalization or KOF Globalization Index

In [ ]:
# Load the KOFGI dataset
kofgi_file_path = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/KOFGI_2023_public.xlsx'
kofgi_data = pd.read_excel(kofgi_file_path)
# Filter the KOFGI data to include only the relevant country codes and years
filtered_kofgi = kofgi_data[(kofgi_data['code'].isin(merged_df['country'])) &
                            (kofgi_data['year'].between(1989, 2008))]
# Select relevant columns
filtered_kofgi = filtered_kofgi[['code', 'year', 'KOFGI']]

# Rename columns to match those in merged_df
filtered_kofgi.rename(columns={'code': 'country', 'year': 'publication_year'}, inplace=True)

# Merge the KOFGI data with merged_df
final_df = merged_df.merge(filtered_kofgi, on=['country', 'publication_year'], how='left')


# Display the first few rows of the final merged DataFrame
print(final_df.head())
final_df

                    sector  publication_year country  total_fraction  \
0  Aircraft_and_spacecraft              1989     AUT             2.0   
1  Aircraft_and_spacecraft              1989     AUS             3.0   
2  Aircraft_and_spacecraft              1989     BEL             1.0   
3  Aircraft_and_spacecraft              1989     CAN             1.0   
4  Aircraft_and_spacecraft              1989     CHE             1.0   

   total_sector_fraction  total_country_fraction       RTA  Normalized_RTA  \
0             17794.9998              16210.8844  0.033268       -0.935607   
1             17794.9998              36588.6751  0.022109       -0.956738   
2             17794.9998              22672.3793  0.011893       -0.976493   
3             17794.9998             106654.5921  0.002528       -0.994956   
4             17794.9998              49316.7954  0.005468       -0.989124   

   Real_GDP_2015_Billion      KOFGI  
0             260.193350  74.134529  
1             510.0367

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA,Real_GDP_2015_Billion,KOFGI
0,Aircraft_and_spacecraft,1989,AUT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607,260.193350,74.134529
1,Aircraft_and_spacecraft,1989,AUS,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738,510.036789,66.225601
2,Aircraft_and_spacecraft,1989,BEL,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493,320.557284,79.794327
3,Aircraft_and_spacecraft,1989,CAN,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956,897.626238,68.699379
4,Aircraft_and_spacecraft,1989,CHE,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124,353.430775,77.953018
...,...,...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JPN,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266,5027.434551,72.535393
2945,Rail_and_transport_equipment,2008,NLD,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201,840.672449,88.188126
2946,Rail_and_transport_equipment,2008,NOR,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482,292.418754,84.463722
2947,Rail_and_transport_equipment,2008,SWE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971,425.330715,88.545303


Create US Dummy Variable

In [ ]:
# Generate US dummy variable
final_df['US_Dummy'] = final_df['country'].apply(lambda x: 1 if x == 'USA' else 0)
print(final_df.head())

                    sector  publication_year country  total_fraction  \
0  Aircraft_and_spacecraft              1989     AUT             2.0   
1  Aircraft_and_spacecraft              1989     AUS             3.0   
2  Aircraft_and_spacecraft              1989     BEL             1.0   
3  Aircraft_and_spacecraft              1989     CAN             1.0   
4  Aircraft_and_spacecraft              1989     CHE             1.0   

   total_sector_fraction  total_country_fraction       RTA  Normalized_RTA  \
0             17794.9998              16210.8844  0.033268       -0.935607   
1             17794.9998              36588.6751  0.022109       -0.956738   
2             17794.9998              22672.3793  0.011893       -0.976493   
3             17794.9998             106654.5921  0.002528       -0.994956   
4             17794.9998              49316.7954  0.005468       -0.989124   

   Real_GDP_2015_Billion      KOFGI  US_Dummy  
0             260.193350  74.134529         0  
1 

In [ ]:

# Save DataFrame to a CSV file
output_file = '/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/final_dataset_1989_2008.csv'
final_df.to_csv(output_file, index=False)

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/final_dataset_1989_2008.csv')
df

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA,Real_GDP_2015_Billion,KOFGI,US_Dummy
0,Aircraft_and_spacecraft,1989,AUT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607,260.193350,74.134529,0
1,Aircraft_and_spacecraft,1989,AUS,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738,510.036789,66.225601,0
2,Aircraft_and_spacecraft,1989,BEL,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493,320.557284,79.794327,0
3,Aircraft_and_spacecraft,1989,CAN,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956,897.626238,68.699379,0
4,Aircraft_and_spacecraft,1989,CHE,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124,353.430775,77.953018,0
...,...,...,...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JPN,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266,5027.434551,72.535393,0
2945,Rail_and_transport_equipment,2008,NLD,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201,840.672449,88.188126,0
2946,Rail_and_transport_equipment,2008,NOR,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482,292.418754,84.463722,0
2947,Rail_and_transport_equipment,2008,SWE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971,425.330715,88.545303,0


In [ ]:
# Load the mapping dataset
import pandas as pd
mapping_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/Patents_IPCs_Final.csv')
mapping_df

,Unnamed: 0,patents_ipcs,patents_cpcs,Consistency,ID,sector,sector_tech,tech_sech
0,1,B64B,B64B,True,1,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
1,2,B64C,B64C,True,2,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
2,3,B64D,B64D,True,3,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
3,4,B64F,B64F,True,4,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
4,5,B64G,B64G,True,5,Aircraft and spacecraft,Aircraft_and_spacecraft,high_tech
...,...,...,...,...,...,...,...,...
444,445,B61L,B61L,True,445,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
445,446,B62M,B62M,True,446,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
446,447,B61G,B61G,True,447,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech
447,448,B62B,B62B,True,448,Rail and transport equipment n.e.c.,Rail_and_transport_equipment,medium_tech


In [ ]:
# Create a dictionary to map sector_tech to tech_sech
sector_to_tech_sech = dict(zip(mapping_df['sector_tech'], mapping_df['tech_sech']))

# Function to map sector to tech_sech
def map_sector_to_tech_sech(sector):
    return sector_to_tech_sech.get(sector, 'Unknown')

# Apply the function to the 'sector' column
df['tech_sech'] = df['sector'].apply(map_sector_to_tech_sech)
df


,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA,Real_GDP_2015_Billion,KOFGI,US_Dummy,tech_sech
0,Aircraft_and_spacecraft,1989,AUT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607,260.193350,74.134529,0,high_tech
1,Aircraft_and_spacecraft,1989,AUS,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738,510.036789,66.225601,0,high_tech
2,Aircraft_and_spacecraft,1989,BEL,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493,320.557284,79.794327,0,high_tech
3,Aircraft_and_spacecraft,1989,CAN,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956,897.626238,68.699379,0,high_tech
4,Aircraft_and_spacecraft,1989,CHE,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124,353.430775,77.953018,0,high_tech
...,...,...,...,...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JPN,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266,5027.434551,72.535393,0,medium_tech
2945,Rail_and_transport_equipment,2008,NLD,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201,840.672449,88.188126,0,medium_tech
2946,Rail_and_transport_equipment,2008,NOR,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482,292.418754,84.463722,0,medium_tech
2947,Rail_and_transport_equipment,2008,SWE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971,425.330715,88.545303,0,medium_tech


Check to see whether the mapping has been execusted correctly.

Result of checking: True. It was correctly mapped.

In [ ]:
# Group by 'sector' and aggregate unique values of 'tech_sech'
mapping_summary = df.groupby('sector')['tech_sech'].unique().reset_index()

# Display the mapping summary
print(mapping_summary)


                                      sector      tech_sech
0                    Aircraft_and_spacecraft    [high_tech]
1        Chemicals_excluding_pharmaceuticals  [medium_tech]
2         Electrical_machinery_and_apparatus  [medium_tech]
3                    Machinery_and_equipment  [medium_tech]
4  Medical_precision_and_optical_instruments    [high_tech]
5  Motor_Vehicles_trailers_and_semi_trailers  [medium_tech]
6  Office_accounting_and_computing_machinery    [high_tech]
7                            Pharmaceuticals    [high_tech]
8      Radio_TV_and_communications_equipment    [high_tech]
9               Rail_and_transport_equipment  [medium_tech]


Add Country Type to the dataset

In [ ]:
# Define the country type mapping
country_type_mapping = {
    'DEU': 'Stable_CME', 'AUT': 'Stable_CME', 'BEL': 'Stable_CME', 'NOR': 'Stable_CME', 'CHE': 'Stable_CME', 'JPN': 'Stable_CME',
    'NLD': 'Liberalized_CME', 'DNK': 'Liberalized_CME', 'FIN': 'Liberalized_CME', 'SWE': 'Liberalized_CME',
    'USA': 'LME', 'GBR': 'LME', 'IRL': 'LME', 'AUS': 'LME', 'CAN': 'LME'
}

# Function to map country to its category
def map_country_to_category(country):
    return country_type_mapping.get(country, 'Unknown')

# Apply the function to the 'country' column to create a new column 'country_category'
df['country_category'] = df['country'].apply(map_country_to_category)
df

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA,Real_GDP_2015_Billion,KOFGI,US_Dummy,tech_sech,country_category
0,Aircraft_and_spacecraft,1989,AUT,2.0000,17794.9998,1.621088e+04,0.033268,-0.935607,260.193350,74.134529,0,high_tech,Stable_CME
1,Aircraft_and_spacecraft,1989,AUS,3.0000,17794.9998,3.658868e+04,0.022109,-0.956738,510.036789,66.225601,0,high_tech,LME
2,Aircraft_and_spacecraft,1989,BEL,1.0000,17794.9998,2.267238e+04,0.011893,-0.976493,320.557284,79.794327,0,high_tech,Stable_CME
3,Aircraft_and_spacecraft,1989,CAN,1.0000,17794.9998,1.066546e+05,0.002528,-0.994956,897.626238,68.699379,0,high_tech,LME
4,Aircraft_and_spacecraft,1989,CHE,1.0000,17794.9998,4.931680e+04,0.005468,-0.989124,353.430775,77.953018,0,high_tech,Stable_CME
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,Rail_and_transport_equipment,2008,JPN,697.9738,45978.9995,1.292078e+06,0.056375,-0.893266,5027.434551,72.535393,0,medium_tech,Stable_CME
2945,Rail_and_transport_equipment,2008,NLD,63.4999,45978.9995,5.207069e+04,0.127268,-0.774201,840.672449,88.188126,0,medium_tech,Liberalized_CME
2946,Rail_and_transport_equipment,2008,NOR,9.2500,45978.9995,7.267207e+03,0.132835,-0.765482,292.418754,84.463722,0,medium_tech,Stable_CME
2947,Rail_and_transport_equipment,2008,SWE,23.7500,45978.9995,4.137875e+04,0.059900,-0.886971,425.330715,88.545303,0,medium_tech,Liberalized_CME


In [ ]:
# Group by 'country' and aggregate unique values of 'country_category' to check the mapping
country_mapping_summary = df.groupby('country')['country_category'].unique().reset_index()

# Display the country mapping summary
print(country_mapping_summary)


   country   country_category
0      AUS              [LME]
1      AUT       [Stable_CME]
2      BEL       [Stable_CME]
3      CAN              [LME]
4      CHE       [Stable_CME]
5      DEU       [Stable_CME]
6      DNK  [Liberalized_CME]
7      FIN  [Liberalized_CME]
8      GBR              [LME]
9      IRL              [LME]
10     JPN       [Stable_CME]
11     NLD  [Liberalized_CME]
12     NOR       [Stable_CME]
13     SWE  [Liberalized_CME]
14     USA              [LME]


Create the four subset datasets based on "publication_year" and "tech_sech" or type of technology

In [ ]:
# Filter the dataset for high tech and years 1991-1998
dataset_high_tech_1991_1998 = df[(df['publication_year'].between(1991, 1998)) & (df['tech_sech'] == 'high_tech')]

# Filter the dataset for high tech and years 2000-2007
dataset_high_tech_2000_2007 = df[(df['publication_year'].between(2000, 2007)) & (df['tech_sech'] == 'high_tech')]

# Filter the dataset for medium tech and years 1991-1998
dataset_medium_tech_1991_1998 = df[(df['publication_year'].between(1991, 1998)) & (df['tech_sech'] == 'medium_tech')]

# Filter the dataset for medium tech and years 2000-2007
dataset_medium_tech_2000_2007 = df[(df['publication_year'].between(2000, 2007)) & (df['tech_sech'] == 'medium_tech')]

# Save the filtered datasets to CSV files
dataset_high_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/dataset_high_tech_1991_1998.csv', index=False)
dataset_high_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/dataset_high_tech_2000_2007.csv', index=False)
dataset_medium_tech_1991_1998.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/dataset_medium_tech_1991_1998.csv', index=False)
dataset_medium_tech_2000_2007.to_csv('/content/drive/My Drive/Colab Notebooks/Patents/Technological_Specialization_BigQuery_IPC/dataset_medium_tech_2000_2007.csv', index=False)
dataset_high_tech_1991_1998
dataset_high_tech_2000_2007
dataset_medium_tech_1991_1998
dataset_medium_tech_2000_2007

,sector,publication_year,country,total_fraction,total_sector_fraction,total_country_fraction,RTA,Normalized_RTA,Real_GDP_2015_Billion,KOFGI,US_Dummy,tech_sech,country_category
418,Chemicals_excluding_pharmaceuticals,2000,AUT,44.9238,418981.9957,1.621088e+04,0.031738,-0.938477,351.116634,85.457169,0,medium_tech,Stable_CME
419,Chemicals_excluding_pharmaceuticals,2000,AUS,71.8728,418981.9957,3.658868e+04,0.022497,-0.955996,721.364640,76.745316,0,medium_tech,LME
420,Chemicals_excluding_pharmaceuticals,2000,BEL,246.6042,418981.9957,2.267238e+04,0.124568,-0.778461,412.018586,86.641289,0,medium_tech,Stable_CME
421,Chemicals_excluding_pharmaceuticals,2000,CAN,302.5157,418981.9957,1.066546e+05,0.032484,-0.937076,1191.853179,81.431488,0,medium_tech,LME
422,Chemicals_excluding_pharmaceuticals,2000,CHE,262.9589,418981.9957,4.931680e+04,0.061066,-0.884898,412.616271,86.896339,0,medium_tech,Stable_CME
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2929,Rail_and_transport_equipment,2007,JPN,707.6667,45978.9995,1.292078e+06,0.057158,-0.891864,5089.747773,71.978432,0,medium_tech,Stable_CME
2930,Rail_and_transport_equipment,2007,NLD,54.6787,45978.9995,5.207069e+04,0.109588,-0.802470,822.814697,88.719330,0,medium_tech,Liberalized_CME
2931,Rail_and_transport_equipment,2007,NOR,15.6667,45978.9995,7.267207e+03,0.224983,-0.632676,291.016290,84.307175,0,medium_tech,Stable_CME
2932,Rail_and_transport_equipment,2007,SWE,33.2500,45978.9995,4.137875e+04,0.083860,-0.845257,429.293624,88.725403,0,medium_tech,Liberalized_CME
